#CAMADA SILVER 

In [0]:
df_customer = spark.table("ifood.ifood_customer_segmentation")
df_orders_rv = spark.table("ifood.ifood_orders_with_cost_revenue")
df_orders = spark.table("ifood.ifood_orders")
df_sessions = spark.table("ifood.ifood_sessions_visits")
df_marketing_push = spark.table("ifood.ifood_marketing_push")
df_marketing_push_full = spark.table("ifood.ifood_marketing_push_full")

In [0]:
import pyspark.sql.functions as F 

In [0]:
df_customer_context = df_customer.filter(F.col('ifood_status') == "Active")
df_customer_context.count()

Out[6]: 103304

In [0]:
df_orders_rv_context = df_orders_rv.filter(F.col('months_after_first_purchase') > "10")
df_orders_rv_context.count()

Out[7]: 354174

In [0]:
df_orders_context = df_orders.filter(F.col('device_app_version') == "MOBILE")
df_orders_context.count()

Out[9]: 178281

In [0]:
df_sessions_context = df_sessions.filter(F.col('platform') == "ANDROID")
df_sessions_context.count()

Out[10]: 200977

In [0]:
df_marketing_push_context = df_marketing_push.filter((F.col('event_name') == "received") 
                                                     & (F.col("event_date") >= "2020-01-01"))
df_marketing_push_context.count()

Out[22]: 2785736

In [0]:
df_marketing_push_full_context = df_marketing_push_full.filter((F.col('event_name') == "send") & (F.col("sample_type") == "control"))
df_marketing_push_full_context.count()

Out[13]: 56695

In [0]:
%sql

create database BB_context

In [0]:
(
  df_customer_context
    .write
    .mode("overwrite")
    .format("delta")
    .option("path", "/FileStore/bb/context/customer_context")
    .saveAsTable("bb_context.customer_context")
)

In [0]:
(
  df_orders_rv_context
    .write
    .mode("overwrite")
    .format("delta")
    .option("path", "/FileStore/bb/context/orders_rv_context")
    .saveAsTable("bb_context.orders_rv_context")
)

In [0]:
(
  df_orders_context
    .write
    .mode("overwrite")
    .format("delta")
    .option("path", "/FileStore/bb/context/orders_context")
    .saveAsTable("bb_context.orders_context")
)

In [0]:
(
  df_sessions_context
    .write
    .mode("overwrite")
    .format("delta")
    .option("path", "/FileStore/bb/context/sessions_context")
    .saveAsTable("bb_context.sessions_context")
)

In [0]:
(
  df_marketing_push_context
    .write
    .mode("overwrite")
    .format("delta")
    .option("path", "/FileStore/bb/context/marketing_push_context")
    .saveAsTable("bb_context.marketing_push_context")
)

In [0]:
(
  df_marketing_push_full_context
    .write
    .mode("overwrite")
    .format("delta")
    .option("path", "/FileStore/bb/context/marketing_push_full_context")
    .saveAsTable("bb_context.marketing_push_full_context")
)

#CAMADA GOLD 

In [0]:
#customer x order-rv
df_app_cs_rv = df_customer_context.join(df_orders_rv_context, on=["customer_id"])
df_app_cs_rv.count()

Out[23]: 2118183

In [0]:
#customer x order
df_app_cs_or = df_customer_context.join(df_orders_context, on=["customer_id"], how="left_anti")
df_app_cs_or.count()

Out[33]: 54549

In [0]:
#session x order
df_app_ss_or = df_sessions_context.join(df_orders_context, on=["session_id"], how="left")
df_app_ss_or.count()

Out[26]: 204938

In [0]:
#customer x mkt
df_app_cs_mkt = (df_customer_context.join(df_marketing_push_context, on=["customer_id"], how="Right")
                 .groupBy(F.col('platform')).count())

df_app_cs_mkt.count()

Out[30]: 2

In [0]:
df_app_cs_mkt.display()

platform,count
android,1740191
ios,1090526


In [0]:
#customer x mkt_full
df_app_cs_mkt_full = (df_customer_context.join(df_marketing_push_full_context, on=["customer_id"], how="Left_semi")
                      .select('customer_id', 'ifood_status', 'orders_last_91d', 'qtt_valid_orders', 'last_nps'))
df_app_cs_mkt_full.count()

Out[28]: 359

In [0]:
df_app_cs_mkt_full.display()

customer_id,segmentation_month,ifood_status_last_month,ifood_status,orders_last_91d,qtt_orders_last_year,qtt_valid_orders,last_valid_order_date,qtt_invalid_orders,last_invalid_order_date,marlin_tag,recency_months,last_nps,registration_date,customer_lifetime_days,customer_lifetime_months,top_3_merchants_code,was_mub_last_month,buyer_last_91d,top_city,top_district,top_centroid_id,first_order_date,last_order_date,days_to_reorder_at_datasource,days_to_reorder_at_concluded,rfv_score,recency_days,recency_days_bucket,recency_days_bucket_description,freq_last_91d,freq_last_91d_bucket,freq_last_91d_bucket_description,avg_aov_last_91d,maturity_orders,maturity_orders_bucket,maturity_orders_bucket_description,benefits_sensitivity,benefits_sensitivity_bucket,preferred_shift_bucket,preferred_shift_bucket_description,merchant_variety,merchant_variety_bucket,merchant_offer,merchant_offer_bucket,merchant_offer_bucket_description,top_dish_bucket,top_dish_bucket_description,preferred_dishes,preferred_dishes_code
40b2121d603d5c1bb769aafd264138637a3dc8f3242ff55ac2268e44673aa9e4,2019-09-01,Resurrected,Active,7.0,13,13,2019-09-29,0,null,1. Marlin,0.0,Promoter,2018-11-04T16:48:57.793+0000,329,10,"[\a9dc1167-af3a-4436-a136-80b4e39fadae""","\""""83c24e10-47ba-48b7-9aee-af66094a56cb\""""","\""""52e6a671-0e0d-47a3-9aa6-4de814e9093e\""""]""""""",1,1,MONTES CLAROS,Conjunto Residencial Jk,-16.7-43.84,2018-11-04,2019-09-29,15.142857142857142,15.142857142857142,2.0,0.0,1.0,1. < 7d,2.31,3.0,3. 2-4,38.96,13.0,4.0,4. 10-49,0.2857142857142857,Baixa,6,null,0.8571428571428571,Alta,349.0,4,4. > 150,1,1. Sem Cozinha Preferida
e1c1d2c00f5cbffc2dbfb87c237fae17bf738d18d01473a8110cc3e00cf22111,2019-12-01,Active,Active,81.0,257,516,2019-12-30,33,2019-12-20,1. Marlin,0.0,Detractor,2015-07-07T23:15:30.000+0000,1637,53,"[\208f095f-0671-4afb-b66d-7c8709ef6079""","\""""e5295625-147e-49f5-8c97-4c989cd8f22d\""""","\""""59866574-5b8e-4361-8c2d-a49bc861129f\""""]""""""",1,1,CURITIBA,Seminario,-25.45-49.3,2015-07-07,2019-12-30,1.0823529411764703,1.1358024691358024,3.0,0.0,1.0,1. < 7d,26.7,5.0,5. > 10,53.05,516.0,5.0,5. 50 ou mais,0.5555555555555556,Media,6,null,0.4814814814814815,Media,2404.0,5,5. > 500,1,1. Sem Cozinha Preferida
75dbdaf18ad3d30c257831b8476a1d6e80f8eb686898fdece14605ccf648441f,2019-08-01,Active,Active,13.0,32,100,2019-08-18,4,2019-03-04,2. Tilapia,0.38709677,Neutral,2016-02-21T21:29:56.000+0000,1286,42,"[\6d193e84-d912-4123-a7e7-d629f247cf36""","\""""dacd21fc-4cde-4bcf-8df9-c8525ee616eb\""""","\""""1da186d9-b196-4db7-ad34-7610ae9f8973\""""]""""""",1,1,AMERICANA,Jardim Terramerica I,-22.77-47.36,2016-03-25,2019-08-18,7.769230769230768,7.769230769230768,3.0,12.0,2.0,2. 7-14d,4.29,4.0,4. 4-10,55.68,100.0,5.0,5. 50 ou mais,0.9230769230769232,Alta,5,5. Jantar,0.7692307692307693,Alta,454.0,4,4. > 150,1,1. Sem Cozinha Preferida
1116cf7c971ee499b0fdf45d724828adc9a9ebb0b0b648353cb22ad773e3b5de,2019-10-01,Active,Active,8.0,27,35,2019-10-24,0,null,1. Marlin,0.25806452,Sem Avaliacoes,2015-02-06T04:39:43.000+0000,621,20,"[\61ffa1af-1e0e-44b8-bcfb-3da2ca91fcc6""","\""""546dae47-3209-4f32-8a70-03149ecaae45\""""","\""""b4aa442c-9487-4311-8fec-dee825f34e8c\""""]""""""",1,1,CAMPINAS,Parque Taquaral,-22.87-47.07,2018-02-18,2019-10-24,17.625,17.625,3.0,8.0,2.0,2. 7-14d,2.64,3.0,3. 2-4,42.28,35.0,4.0,4. 10-49,0.25,Baixa,6,null,0.625,Media,1487.0,5,5. > 500,3,3. Mais de uma Cozinha Preferida
c7e9b50fb34921e5efba7fee2099ce42c633bfc0a1b7d8e46b8705b0a6f740d5,2019-07-01,Active,Active,9.0,28,35,2019-07-04,1,2019-03-09,4. Retention Carp,0.83870968,Promoter,2018-05-05T08:56:35.082+0000,451,14,"[\5f6ece10-bc65-4782-bc7a-eb712c4d213b""","\""""1aa6388d-6ae8-4243-aa9d-314ae301b0b6\""""","\""""39137d97-8ab1-49cd-be38-4a179c2fe01d\""""]""""""",1,1,RIO DE JANEIRO,Flamengo,-22.94-43.18,2018-05-05,2019-07-04,12.888888888888891,12.888888888888891,3.0,26.0,3.0,3. 14-28d,2.97,3.0,3. 2-4,76.28,35.0,4.0,4. 10-49,0.6666666666666666,Alta,5,5. Jantar,1.0,Pedido Unico,1765.0,5,5. > 500,1,1. Sem Cozinha Preferid

In [0]:
%sql

create database bb_app

In [0]:
(
  df_app_cs_rv
    .write
    .mode("append")
    .format("delta")
    .option("path", "/FileStore/bb/app/customer_orders_rv")
    .saveAsTable("bb_app.customer_orders_rv")
)

In [0]:
(
  df_app_ss_or
    .write
    .mode("append")
    .format("delta")
    .option("path", "/FileStore/bb/app/session_order")
    .saveAsTable("bb_app.session_order")
)

In [0]:
(
  df_app_cs_mkt
    .write
    .mode("append")
    .format("delta")
    .option("path", "/FileStore/bb/app/customer_mkt")
    .saveAsTable("bb_app.customer_mkt")
)

In [0]:
(
  df_app_cs_mkt_full
    .write
    .mode("append")
    .format("delta")
    .option("path", "/FileStore/bb/app/customer_mkt_full")
    .saveAsTable("bb_app.customer_mkt_full")
)